In [1]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [2]:
df = pd.read_csv("IMDB_Dataset.zip")

In [3]:
df.shape

(50000, 2)

In [4]:
df = df[:10000]

In [5]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
df.shape

(10000, 2)

In [7]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [8]:
from bs4 import BeautifulSoup

In [9]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text(separator=" ", strip=True)

In [10]:
df["review"] = tqdm(df["review"].apply(remove_html_tags))

/tmp/ipykernel_12423/3157248265.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


  0%|          | 0/10000 [00:00<?, ?it/s]

In [11]:
df["review"][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me. The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word. It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away. I would say the main appeal of the show is due to the fact that it goes where other shows

In [12]:
import string

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [13]:
# !python -m spacy download en_core_web_md
import spacy

nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

In [14]:
def text_preprocessing(text):
    return [
        [token.lemma_ for token in doc if not token.is_punct and not token.is_stop]
        for doc in tqdm(
            nlp.pipe(
                text.astype(str).str.lower().str.strip().tolist(), batch_size=1000
            ),
            total=len(text),
        )
    ]

In [15]:
df["review"] = text_preprocessing(df["review"])

  0%|          | 0/10000 [00:00<?, ?it/s]

In [16]:
df.head()

,review,sentiment
0,"[reviewer, mention, watch, 1, oz, episode, hoo...",positive
1,"[wonderful, little, production, filming, techn...",positive
2,"[think, wonderful, way, spend, time, hot, summ...",positive
3,"[basically, family, little, boy, jake, think, ...",negative
4,"[petter, mattei, love, time, money, visually, ...",positive


In [17]:
sentence_lists = df["review"].tolist()
print(sentence_lists[:2])

[['reviewer', 'mention', 'watch', '1', 'oz', 'episode', 'hook', 'right', 'exactly', 'happen', 'thing', 'strike', 'oz', 'brutality', 'unflinche', 'scene', 'violence', 'set', 'right', 'word', 'trust', 'faint', 'hearted', 'timid', 'pull', 'punch', 'regard', 'drug', 'sex', 'violence', 'hardcore', 'classic', 'use', 'word', 'call', 'oz', 'nickname', 'give', 'oswald', 'maximum', 'security', 'state', 'penitentary', 'focus', 'mainly', 'emerald', 'city', 'experimental', 'section', 'prison', 'cell', 'glass', 'front', 'face', 'inward', 'privacy', 'high', 'agenda', 'em', 'city', 'home', 'aryan', 'muslim', 'gangsta', 'latinos', 'christians', 'italian', 'irish', 'scuffle', 'death', 'stare', 'dodgy', 'dealing', 'shady', 'agreement', 'far', 'away', 'main', 'appeal', 'fact', 'go', 'show', 'dare', 'forget', 'pretty', 'picture', 'paint', 'mainstream', 'audience', 'forget', 'charm', 'forget', 'romance', 'oz', 'mess', 'episode', 'see', 'strike', 'nasty', 'surreal', 'ready', 'watch', 'develop', 'taste', 'oz'

In [18]:
def tokens_into_strings(token_lists):
    return [" ".join(lst) for lst in token_lists]

In [19]:
tokens_strings = tokens_into_strings(df["review"])
tokens_strings[:5]

['reviewer mention watch 1 oz episode hook right exactly happen thing strike oz brutality unflinche scene violence set right word trust faint hearted timid pull punch regard drug sex violence hardcore classic use word call oz nickname give oswald maximum security state penitentary focus mainly emerald city experimental section prison cell glass front face inward privacy high agenda em city home aryan muslim gangsta latinos christians italian irish scuffle death stare dodgy dealing shady agreement far away main appeal fact go show dare forget pretty picture paint mainstream audience forget charm forget romance oz mess episode see strike nasty surreal ready watch develop taste oz get accustom high level graphic violence violence injustice crooked guard sell nickel inmate kill order away mannered middle class inmate turn prison bitch lack street skill prison experience watch oz comfortable uncomfortable view that touch dark',
 'wonderful little production filming technique unassuming- old

# Applying text representation techniques:

## Tfidf vectorization:

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
df_vecs = tfidf.fit_transform(tokens_strings)

In [21]:
print(df_vecs[:1, :])

  (0, 32190)	0.06288074908836157
  (0, 24592)	0.04900345506642814
  (0, 41790)	0.07520229196016928
  (0, 27951)	0.4911919661627683
  (0, 12805)	0.09871143458569412
  (0, 18264)	0.06802132824254413
  (0, 32363)	0.07899293270684586
  (0, 13167)	0.054497507593535256
  (0, 17231)	0.03960520166543676
  (0, 38391)	0.030293902110014037
  (0, 36797)	0.12890106714262967
  (0, 5439)	0.08646353561999769
  (0, 40179)	0.11393869098948975
  (0, 33570)	0.029145077371737906
  (0, 41319)	0.21770339389715013
  (0, 34242)	0.03943167614584476
  (0, 42611)	0.09155255657732388
  (0, 39444)	0.06690683151533683
  (0, 13563)	0.08742589358031894
  (0, 17558)	0.07140847925447974
  (0, 38624)	0.10020111330474372
  (0, 30545)	0.053730976265917746
  (0, 30569)	0.0708647204511568
  (0, 31590)	0.06681027132827676
  (0, 11654)	0.058239385012031256
  :	:
  (0, 22368)	0.05288178181175004
  (0, 16426)	0.06643131862835731
  (0, 19462)	0.08902438368454765
  (0, 9049)	0.08902438368454765
  (0, 16741)	0.07168949067793652
  (

In [22]:
df_tfidf = pd.DataFrame.sparse.from_spmatrix(
    df_vecs, columns=tfidf.get_feature_names_out()
)

In [23]:
df_tfidf.isnull().sum()

00            False
000           False
00001         False
0069          False
007           False
              ...  
óli           False
önsjön        False
über          False
überwoman     False
ünfaithful    False
Length: 43349, dtype: Sparse[bool, False]

In [24]:
x = 0
print(
    f"\nNon zero values in row no. {x}:\n",
    df_tfidf.iloc[x].sort_values(ascending=False).head(),
)


Non zero values in row no. 0:
 oz          0.491192
violence    0.217703
prison      0.201307
inmate      0.187627
forget      0.152219
Name: 0, dtype: Sparse[float64, 0]


In [25]:
y = df["sentiment"]
y.head()

0    positive
1    positive
2    positive
3    negative
4    positive
Name: sentiment, dtype: object

In [26]:
y.shape

(10000,)

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [28]:
x_train, x_test, y_train, y_test = train_test_split(df_tfidf, y, test_size=0.2, random_state=False)

In [29]:
rmf = RandomForestClassifier(n_jobs=-1)
nb = MultinomialNB()

In [30]:
rmf.fit(x_train, y_train)
nb.fit(x_train, y_train)
y_pred1 = rmf.predict(x_test)
y_pred2 = nb.predict(x_test)

In [31]:
print("Random_forest: ", accuracy_score(y_test, y_pred1) * 100)
print("Naive_bayse: ", accuracy_score(y_test, y_pred2) * 100)

Random_forest:  84.85000000000001
Naive_bayse:  84.45


## word2vec:

In [32]:
from gensim.models import Word2Vec

In [33]:
w2v = Word2Vec(sentences=sentence_lists, window=2, vector_size=100, workers=4)

In [34]:
w2v.save("word2vec_on_IMDB.model")

In [35]:
def document_vector(doc, model):
    # Remove out-of-vocabulary words
    words = [word for word in doc if word in model.wv.key_to_index]
    return np.mean([model.wv[word] for word in words], axis=0)

In [36]:
x = np.array([document_vector(doc, w2v) for doc in sentence_lists])

In [37]:
x.shape

(10000, 100)

In [38]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=False)

In [39]:
rmf = RandomForestClassifier(n_jobs=-1)
nb = MultinomialNB()

In [40]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_train_nonneg = scaler.fit_transform(x_train)  # Now all values >= 0
x_test_nonneg = scaler.fit_transform(x_test)  # Now all values >= 0

In [41]:
rmf.fit(x_train, y_train)
nb.fit(x_train_nonneg, y_train)
y_pred1 = rmf.predict(x_test)
y_pred2 = nb.predict(x_test)

In [42]:
print("Random_forest: ", accuracy_score(y_test, y_pred1) * 100)
print("Naive_bayse: ", accuracy_score(y_test, y_pred2) * 100)

Random_forest:  77.0
Naive_bayse:  49.15


## Bag of words:

In [43]:
from sklearn.feature_extraction.text import CountVectorizer

In [44]:
cv = CountVectorizer()
vectors_bow = cv.fit_transform(tokens_strings)

In [45]:
print(vectors_bow[0])

  (0, 32190)	1
  (0, 24592)	1
  (0, 41790)	3
  (0, 27951)	6
  (0, 12805)	2
  (0, 18264)	1
  (0, 32363)	2
  (0, 13167)	1
  (0, 17231)	1
  (0, 38391)	1
  (0, 36797)	2
  (0, 5439)	1
  (0, 40179)	1
  (0, 33570)	1
  (0, 41319)	4
  (0, 34242)	1
  (0, 42611)	2
  (0, 39444)	1
  (0, 13563)	1
  (0, 17558)	1
  (0, 38624)	1
  (0, 30545)	1
  (0, 30569)	1
  (0, 31590)	1
  (0, 11654)	1
  :	:
  (0, 22368)	1
  (0, 16426)	1
  (0, 19462)	1
  (0, 9049)	1
  (0, 16741)	1
  (0, 34061)	1
  (0, 26595)	1
  (0, 19469)	2
  (0, 21262)	1
  (0, 27544)	1
  (0, 23682)	1
  (0, 24799)	1
  (0, 7387)	1
  (0, 39555)	1
  (0, 4322)	1
  (0, 21763)	1
  (0, 36759)	1
  (0, 35127)	1
  (0, 13327)	1
  (0, 7860)	1
  (0, 39908)	1
  (0, 41237)	1
  (0, 38282)	1
  (0, 38968)	1
  (0, 9587)	1


In [46]:
df_bow = pd.DataFrame.sparse.from_spmatrix(vectors_bow, columns=cv.get_feature_names_out())

In [47]:
df_bow.head()

,00,000,00001,0069,007,00am,01,0126,01pm,02,...,être,ís,ísnt,île,ïn,óli,önsjön,über,überwoman,ünfaithful
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
df_bow.shape

(10000, 43349)

In [49]:
x_train, x_test, y_train, y_test = train_test_split(df_bow, y, test_size=0.2, random_state=False)

In [50]:
rmf = RandomForestClassifier(n_jobs=-1)
nb = MultinomialNB()

In [51]:
rmf.fit(x_train, y_train)
nb.fit(x_train, y_train)
y_pred1 = rmf.predict(x_test)
y_pred2 = nb.predict(x_test)

In [52]:
print("Random_forest: ", accuracy_score(y_test, y_pred1) * 100)
print("Naive_bayse: ", accuracy_score(y_test, y_pred2) * 100)

Random_forest:  85.1
Naive_bayse:  83.3
